# Lab: Infrastructure as Code with Terraform & Ansible

## Overview

In this hands-on lab, you will:
- Install and use Terraform for infrastructure provisioning
- Learn Terraform workflow: init, plan, apply, destroy
- Manage infrastructure state and variables
- Install and use Ansible for configuration management
- Write Ansible playbooks to configure systems
- Integrate Terraform and Ansible for complete IaC workflows

## Prerequisites

- Ubuntu 22.04 VM (fresh installation)
- Terminal access to the VM
- Basic familiarity with Linux commands and YAML

## Time Required

Approximately 90-120 minutes

---

## Part 1: Environment Setup

### 1.1 Update the System

In [ ]:
%%bash
sudo apt-get update && sudo apt-get upgrade -y

### 1.2 Install Terraform

We'll install Terraform from HashiCorp's official repository.

In [ ]:
%%bash
# Install dependencies
sudo apt-get install -y gnupg software-properties-common curl

# Add HashiCorp GPG key
wget -O- https://apt.releases.hashicorp.com/gpg | \
    gpg --dearmor | \
    sudo tee /usr/share/keyrings/hashicorp-archive-keyring.gpg > /dev/null

# Add HashiCorp repository
echo "deb [signed-by=/usr/share/keyrings/hashicorp-archive-keyring.gpg] \
    https://apt.releases.hashicorp.com $(lsb_release -cs) main" | \
    sudo tee /etc/apt/sources.list.d/hashicorp.list

# Install Terraform
sudo apt-get update
sudo apt-get install -y terraform

# Verify installation
terraform version

In [ ]:
%%bash
# Install Python and pip
sudo apt-get install -y python3 python3-pip sshpass

# Install Ansible system-wide (needed for sudo with Docker)
sudo pip3 install ansible

# Verify installation
ansible --version | head -5

In [ ]:
%%bash
# Install Python and pip
sudo apt-get install -y python3 python3-pip sshpass

# Install Ansible
pip3 install ansible

# Add to PATH if needed
export PATH="$HOME/.local/bin:$PATH"
echo 'export PATH="$HOME/.local/bin:$PATH"' >> ~/.bashrc

# Verify installation
~/.local/bin/ansible --version | head -5

### 1.4 Install Docker (for later exercises)

We'll use Docker to simulate infrastructure that Terraform can provision.

In [ ]:
%%bash
# Install Docker
curl -fsSL https://get.docker.com | sudo sh

# Add current user to docker group
sudo usermod -aG docker $USER

# Start Docker service
sudo systemctl enable docker
sudo systemctl start docker

# Verify (using sudo for now, group membership takes effect on next login)
sudo docker --version

### 1.5 Create Working Directory

In [ ]:
%%bash
mkdir -p ~/iac-lab
cd ~/iac-lab
echo "Working directory: $(pwd)"

---

## Part 2: Terraform Fundamentals

### 2.1 Understanding HCL (HashiCorp Configuration Language)

Terraform uses HCL, a declarative language for defining infrastructure.

**Key concepts:**
- **Blocks**: Containers for configuration (resource, variable, output)
- **Arguments**: Settings within blocks (name = value)
- **Expressions**: Values that can be computed

### 2.2 Your First Terraform Configuration

Let's create a simple configuration using the `local` provider to create files.

In [ ]:
%%bash
mkdir -p ~/iac-lab/terraform-basics
cd ~/iac-lab/terraform-basics

# Create main.tf
cat > main.tf << 'EOF'
# Terraform configuration block
terraform {
  required_version = ">= 1.0.0"
  
  required_providers {
    local = {
      source  = "hashicorp/local"
      version = "~> 2.0"
    }
    random = {
      source  = "hashicorp/random"
      version = "~> 3.0"
    }
  }
}

# Generate a random ID
resource "random_id" "server_id" {
  byte_length = 4
}

# Create a local file
resource "local_file" "config" {
  filename = "${path.module}/output/server-config.txt"
  content  = <<-EOT
    Server Configuration
    ====================
    Server ID: ${random_id.server_id.hex}
    Generated: ${timestamp()}
    Environment: development
  EOT
  
  # Ensure output directory exists
  depends_on = [null_resource.create_output_dir]
}

# Create output directory
resource "null_resource" "create_output_dir" {
  provisioner "local-exec" {
    command = "mkdir -p ${path.module}/output"
  }
}

# Output the server ID
output "server_id" {
  value       = random_id.server_id.hex
  description = "The generated server ID"
}

output "config_file_path" {
  value       = local_file.config.filename
  description = "Path to the generated config file"
}
EOF

echo "Created main.tf:"
cat main.tf

### 2.3 Terraform Workflow: Init

The `terraform init` command initializes the working directory and downloads required providers.

In [ ]:
%%bash
cd ~/iac-lab/terraform-basics
terraform init

### 2.4 Terraform Workflow: Plan

The `terraform plan` command shows what changes Terraform will make without actually making them.

In [ ]:
%%bash
cd ~/iac-lab/terraform-basics
terraform plan

### 2.5 Terraform Workflow: Apply

The `terraform apply` command executes the planned changes.

In [ ]:
%%bash
cd ~/iac-lab/terraform-basics
terraform apply -auto-approve

### 2.6 Verify the Results

In [ ]:
%%bash
cd ~/iac-lab/terraform-basics

echo "=== Generated Files ==="
ls -la output/

echo ""
echo "=== Config File Contents ==="
cat output/server-config.txt

echo ""
echo "=== Terraform Outputs ==="
terraform output

---

## Part 3: Terraform Variables and Outputs

### 3.1 Create a Configuration with Variables

In [ ]:
%%bash
mkdir -p ~/iac-lab/terraform-variables
cd ~/iac-lab/terraform-variables

# Create variables.tf
cat > variables.tf << 'EOF'
variable "environment" {
  description = "Deployment environment"
  type        = string
  default     = "development"
}

variable "app_name" {
  description = "Application name"
  type        = string
  default     = "myapp"
}

variable "instance_count" {
  description = "Number of instances to create"
  type        = number
  default     = 2
}

variable "tags" {
  description = "Resource tags"
  type        = map(string)
  default = {
    project = "iac-lab"
    owner   = "devops-team"
  }
}

variable "allowed_ports" {
  description = "List of allowed ports"
  type        = list(number)
  default     = [22, 80, 443]
}
EOF

echo "Created variables.tf"

In [ ]:
%%bash
cd ~/iac-lab/terraform-variables

# Create main.tf that uses variables
cat > main.tf << 'EOF'
terraform {
  required_providers {
    local = {
      source  = "hashicorp/local"
      version = "~> 2.0"
    }
  }
}

# Create configuration files for each instance
resource "local_file" "instance_config" {
  count    = var.instance_count
  filename = "${path.module}/configs/${var.app_name}-${var.environment}-${count.index + 1}.json"
  content  = jsonencode({
    instance_id  = count.index + 1
    app_name     = var.app_name
    environment  = var.environment
    tags         = var.tags
    allowed_ports = var.allowed_ports
    hostname     = "${var.app_name}-${var.environment}-${count.index + 1}"
  })

  provisioner "local-exec" {
    command = "mkdir -p ${path.module}/configs"
  }
}

# Create a summary file
resource "local_file" "summary" {
  filename = "${path.module}/configs/deployment-summary.txt"
  content  = <<-EOT
    Deployment Summary
    ==================
    Application: ${var.app_name}
    Environment: ${var.environment}
    Instance Count: ${var.instance_count}
    Allowed Ports: ${join(", ", var.allowed_ports)}
    Tags:
    %{for key, value in var.tags~}
      - ${key}: ${value}
    %{endfor~}
  EOT

  depends_on = [local_file.instance_config]
}
EOF

echo "Created main.tf"

In [ ]:
%%bash
cd ~/iac-lab/terraform-variables

# Create outputs.tf
cat > outputs.tf << 'EOF'
output "instance_files" {
  description = "List of created instance config files"
  value       = local_file.instance_config[*].filename
}

output "deployment_info" {
  description = "Deployment information"
  value = {
    app_name     = var.app_name
    environment  = var.environment
    instances    = var.instance_count
  }
}
EOF

echo "Created outputs.tf"

### 3.2 Create a terraform.tfvars File

Override default values using a variables file.

In [ ]:
%%bash
cd ~/iac-lab/terraform-variables

# Create terraform.tfvars
cat > terraform.tfvars << 'EOF'
environment    = "staging"
app_name       = "webserver"
instance_count = 3

tags = {
  project     = "iac-lab"
  owner       = "devops-team"
  cost_center = "engineering"
}

allowed_ports = [22, 80, 443, 8080]
EOF

echo "Created terraform.tfvars:"
cat terraform.tfvars

### 3.3 Apply the Configuration

In [ ]:
%%bash
cd ~/iac-lab/terraform-variables

terraform init
terraform apply -auto-approve

In [ ]:
%%bash
cd ~/iac-lab/terraform-variables

echo "=== Created Files ==="
ls -la configs/

echo ""
echo "=== Instance Config Example ==="
cat configs/webserver-staging-1.json | python3 -m json.tool

echo ""
echo "=== Deployment Summary ==="
cat configs/deployment-summary.txt

---

## Part 4: Terraform State Management

### 4.1 Examine the State File

In [ ]:
%%bash
cd ~/iac-lab/terraform-variables

echo "=== State File Info ==="
ls -la terraform.tfstate

echo ""
echo "=== State Contents (first 50 lines) ==="
head -50 terraform.tfstate

### 4.2 Terraform State Commands

In [ ]:
%%bash
cd ~/iac-lab/terraform-variables

echo "=== List Resources in State ==="
terraform state list

echo ""
echo "=== Show Specific Resource ==="
terraform state show 'local_file.instance_config[0]'

In [ ]:
%%bash
cd ~/iac-lab/terraform-variables

echo "=== Terraform Show (Human-Readable State) ==="
terraform show | head -60

---

## Part 5: Ansible Fundamentals

### 5.1 Create Ansible Working Directory

In [ ]:
%%bash
mkdir -p ~/iac-lab/ansible-basics
cd ~/iac-lab/ansible-basics
echo "Working directory: $(pwd)"

### 5.2 Create Ansible Inventory

The inventory file defines the hosts Ansible will manage.

In [ ]:
%%bash
cd ~/iac-lab/ansible-basics

# Create inventory file for localhost
cat > inventory.ini << 'EOF'
[local]
localhost ansible_connection=local

[webservers]
localhost ansible_connection=local

[all:vars]
ansible_python_interpreter=/usr/bin/python3
EOF

echo "Created inventory.ini:"
cat inventory.ini

### 5.3 Ansible Ad-Hoc Commands

Ad-hoc commands let you run quick tasks without writing a playbook.

In [ ]:
%%bash
cd ~/iac-lab/ansible-basics
export PATH="$HOME/.local/bin:$PATH"

echo "=== Ping all hosts ==="
ansible -i inventory.ini all -m ping

echo ""
echo "=== Gather facts from localhost ==="
ansible -i inventory.ini localhost -m setup -a 'filter=ansible_distribution*' 2>/dev/null

In [ ]:
%%bash
cd ~/iac-lab/ansible-basics
export PATH="$HOME/.local/bin:$PATH"

echo "=== Run shell command ==="
ansible -i inventory.ini localhost -m shell -a 'hostname && uptime'

echo ""
echo "=== Check disk space ==="
ansible -i inventory.ini localhost -m shell -a 'df -h /'

### 5.4 Your First Ansible Playbook

Playbooks are YAML files that define automation tasks.

In [ ]:
%%bash
cd ~/iac-lab/ansible-basics

cat > setup-playbook.yml << 'EOF'
---
- name: Basic System Setup
  hosts: localhost
  connection: local
  become: no
  
  vars:
    app_name: "myapp"
    app_dir: "/tmp/{{ app_name }}"
    config_files:
      - name: "app.conf"
        content: "# Application configuration\napp_name={{ app_name }}\ndebug=false"
      - name: "logging.conf"
        content: "# Logging configuration\nlog_level=INFO\nlog_file=/var/log/{{ app_name }}.log"
  
  tasks:
    - name: Display start message
      debug:
        msg: "Starting setup for {{ app_name }}"
    
    - name: Create application directory
      file:
        path: "{{ app_dir }}"
        state: directory
        mode: '0755'
    
    - name: Create subdirectories
      file:
        path: "{{ app_dir }}/{{ item }}"
        state: directory
        mode: '0755'
      loop:
        - config
        - logs
        - data
    
    - name: Create configuration files
      copy:
        content: "{{ item.content }}"
        dest: "{{ app_dir }}/config/{{ item.name }}"
        mode: '0644'
      loop: "{{ config_files }}"
    
    - name: Create version file
      copy:
        content: |
          Application: {{ app_name }}
          Version: 1.0.0
          Deployed: {{ ansible_date_time.iso8601 }}
          Host: {{ ansible_hostname }}
        dest: "{{ app_dir }}/VERSION"
        mode: '0644'
    
    - name: Display completion message
      debug:
        msg: "Setup complete! Files created in {{ app_dir }}"
EOF

echo "Created setup-playbook.yml"

### 5.5 Run the Playbook

In [ ]:
%%bash
cd ~/iac-lab/ansible-basics
export PATH="$HOME/.local/bin:$PATH"

ansible-playbook -i inventory.ini setup-playbook.yml

In [ ]:
%%bash
echo "=== Verify Created Structure ==="
find /tmp/myapp -type f -o -type d | head -20

echo ""
echo "=== VERSION File ==="
cat /tmp/myapp/VERSION

echo ""
echo "=== App Config ==="
cat /tmp/myapp/config/app.conf

---

## Part 6: Advanced Ansible Playbooks

### 6.1 Playbook with Handlers and Templates

In [ ]:
%%bash
cd ~/iac-lab/ansible-basics
mkdir -p templates

# Create a Jinja2 template
cat > templates/nginx.conf.j2 << 'EOF'
# Nginx configuration generated by Ansible
# Generated on: {{ ansible_date_time.iso8601 }}

server {
    listen {{ http_port | default(80) }};
    server_name {{ server_name | default('localhost') }};
    
    root {{ document_root }};
    index index.html;
    
    # Logging
    access_log /var/log/nginx/{{ server_name }}_access.log;
    error_log /var/log/nginx/{{ server_name }}_error.log;
    
    location / {
        try_files $uri $uri/ =404;
    }
    
{% if enable_php | default(false) %}
    location ~ \.php$ {
        fastcgi_pass unix:/var/run/php/php-fpm.sock;
        fastcgi_index index.php;
        include fastcgi_params;
    }
{% endif %}
}
EOF

echo "Created nginx.conf.j2 template"

In [ ]:
%%bash
cd ~/iac-lab/ansible-basics

cat > webserver-playbook.yml << 'EOF'
---
- name: Configure Web Server
  hosts: localhost
  connection: local
  become: no
  
  vars:
    server_name: "app.example.com"
    http_port: 8080
    document_root: "/var/www/myapp"
    enable_php: false
    config_dir: "/tmp/nginx-configs"
  
  handlers:
    - name: reload nginx
      debug:
        msg: "Handler triggered: Would reload nginx service"
  
  tasks:
    - name: Create config directory
      file:
        path: "{{ config_dir }}"
        state: directory
        mode: '0755'
    
    - name: Generate nginx configuration from template
      template:
        src: templates/nginx.conf.j2
        dest: "{{ config_dir }}/{{ server_name }}.conf"
        mode: '0644'
      notify: reload nginx
    
    - name: Create document root
      file:
        path: "{{ document_root }}"
        state: directory
        mode: '0755'
      ignore_errors: yes
    
    - name: Create index.html
      copy:
        content: |
          <!DOCTYPE html>
          <html>
          <head><title>{{ server_name }}</title></head>
          <body>
            <h1>Welcome to {{ server_name }}</h1>
            <p>Configured by Ansible</p>
            <p>Port: {{ http_port }}</p>
          </body>
          </html>
        dest: "{{ config_dir }}/index.html"
        mode: '0644'
    
    - name: Display configuration summary
      debug:
        msg:
          - "Server: {{ server_name }}"
          - "Port: {{ http_port }}"
          - "Document Root: {{ document_root }}"
          - "Config written to: {{ config_dir }}"
EOF

echo "Created webserver-playbook.yml"

In [ ]:
%%bash
cd ~/iac-lab/ansible-basics
export PATH="$HOME/.local/bin:$PATH"

ansible-playbook -i inventory.ini webserver-playbook.yml

In [ ]:
%%bash
echo "=== Generated Nginx Config ==="
cat /tmp/nginx-configs/app.example.com.conf

echo ""
echo "=== Generated Index Page ==="
cat /tmp/nginx-configs/index.html

---

## Part 7: Terraform + Ansible Integration

Now we'll combine Terraform and Ansible: Terraform provisions infrastructure, Ansible configures it.

### 7.1 Set Up the Integration Project

In [ ]:
%%bash
mkdir -p ~/iac-lab/terraform-ansible-integration/{terraform,ansible}
cd ~/iac-lab/terraform-ansible-integration
echo "Project structure:"
find . -type d

### 7.2 Create Terraform Configuration for Docker Containers

We'll use Terraform to create Docker containers that simulate servers.

In [ ]:
%%bash
cd ~/iac-lab/terraform-ansible-integration/terraform

cat > main.tf << 'EOF'
terraform {
  required_providers {
    docker = {
      source  = "kreuzwerker/docker"
      version = "~> 3.0"
    }
    local = {
      source  = "hashicorp/local"
      version = "~> 2.0"
    }
  }
}

provider "docker" {
  host = "unix:///var/run/docker.sock"
}

variable "container_count" {
  description = "Number of containers to create"
  type        = number
  default     = 2
}

variable "base_port" {
  description = "Base port for container mapping"
  type        = number
  default     = 8000
}

# Pull Ubuntu image
resource "docker_image" "ubuntu" {
  name         = "ubuntu:22.04"
  keep_locally = true
}

# Create containers
resource "docker_container" "webserver" {
  count = var.container_count
  
  name  = "webserver-${count.index + 1}"
  image = docker_image.ubuntu.image_id
  
  # Keep container running
  command = ["tail", "-f", "/dev/null"]
  
  # Enable for SSH-like access
  stdin_open = true
  tty        = true
  
  ports {
    internal = 80
    external = var.base_port + count.index
  }
  
  labels {
    label = "managed_by"
    value = "terraform"
  }
  
  labels {
    label = "environment"
    value = "lab"
  }
}

# Generate Ansible inventory from Terraform
resource "local_file" "ansible_inventory" {
  filename = "${path.module}/../ansible/inventory.ini"
  content  = <<-EOT
# Ansible inventory generated by Terraform
# Generated: ${timestamp()}

[webservers]
%{for idx, container in docker_container.webserver~}
${container.name} ansible_connection=docker
%{endfor~}

[webservers:vars]
ansible_python_interpreter=/usr/bin/python3

[all:vars]
environment=lab
managed_by=terraform
  EOT
}

# Output container information
output "container_names" {
  value = docker_container.webserver[*].name
}

output "container_ports" {
  value = [
    for idx, container in docker_container.webserver : {
      name = container.name
      port = var.base_port + idx
    }
  ]
}

output "inventory_file" {
  value = local_file.ansible_inventory.filename
}
EOF

echo "Created Terraform configuration"

### 7.3 Apply Terraform to Create Containers

In [ ]:
%%bash
cd ~/iac-lab/terraform-ansible-integration/terraform

terraform init

In [ ]:
%%bash
cd ~/iac-lab/terraform-ansible-integration/terraform

# Need sudo for Docker socket access
sudo terraform apply -auto-approve

In [ ]:
%%bash
echo "=== Running Containers ==="
sudo docker ps --filter "label=managed_by=terraform" --format "table {{.Names}}\t{{.Status}}\t{{.Ports}}"

echo ""
echo "=== Generated Ansible Inventory ==="
cat ~/iac-lab/terraform-ansible-integration/ansible/inventory.ini

### 7.4 Create Ansible Playbook to Configure Containers

In [ ]:
%%bash
cd ~/iac-lab/terraform-ansible-integration/ansible

cat > configure-webservers.yml << 'EOF'
---
- name: Configure Web Servers
  hosts: webservers
  gather_facts: no
  
  vars:
    packages:
      - python3
      - nginx
      - curl
  
  tasks:
    - name: Update apt cache
      raw: apt-get update
      changed_when: false
    
    - name: Install Python (required for Ansible modules)
      raw: apt-get install -y python3
      changed_when: false
    
    - name: Gather facts after Python is installed
      setup:
    
    - name: Install packages
      apt:
        name: "{{ packages }}"
        state: present
        update_cache: yes
    
    - name: Create web directory
      file:
        path: /var/www/html
        state: directory
        mode: '0755'
    
    - name: Create custom index page
      copy:
        content: |
          <!DOCTYPE html>
          <html>
          <head><title>{{ inventory_hostname }}</title></head>
          <body>
            <h1>Welcome to {{ inventory_hostname }}</h1>
            <p>This server was:</p>
            <ul>
              <li>Provisioned by: Terraform</li>
              <li>Configured by: Ansible</li>
              <li>OS: {{ ansible_distribution }} {{ ansible_distribution_version }}</li>
            </ul>
            <p>Timestamp: {{ ansible_date_time.iso8601 }}</p>
          </body>
          </html>
        dest: /var/www/html/index.html
        mode: '0644'
    
    - name: Create server info file
      copy:
        content: |
          hostname={{ inventory_hostname }}
          managed_by={{ managed_by | default('ansible') }}
          environment={{ environment | default('unknown') }}
          configured_at={{ ansible_date_time.iso8601 }}
        dest: /etc/server-info
        mode: '0644'
    
    - name: Display completion message
      debug:
        msg: "Configuration complete for {{ inventory_hostname }}"
EOF

echo "Created configure-webservers.yml"

### 7.5 Run Ansible to Configure the Containers

In [ ]:
%%bash
cd ~/iac-lab/terraform-ansible-integration/ansible
export PATH="$HOME/.local/bin:$PATH"

# Run Ansible with sudo (needed for Docker connection)
sudo ~/.local/bin/ansible-playbook -i inventory.ini configure-webservers.yml

### 7.6 Verify the Configuration

In [ ]:
%%bash
echo "=== Check webserver-1 ==="
sudo docker exec webserver-1 cat /var/www/html/index.html

echo ""
echo "=== Server Info ==="
sudo docker exec webserver-1 cat /etc/server-info

echo ""
echo "=== Installed Packages ==="
sudo docker exec webserver-1 dpkg -l | grep -E "nginx|python3 " | head -5

In [ ]:
%%bash
echo "=== Check webserver-2 ==="
sudo docker exec webserver-2 cat /etc/server-info

---

## Part 8: Version Control for Infrastructure Code

### 8.1 Create .gitignore Files

In [ ]:
%%bash
cd ~/iac-lab

# Create Terraform .gitignore
cat > .gitignore << 'EOF'
# Terraform
**/.terraform/
**/*.tfstate
**/*.tfstate.*
**/*.tfvars
!example.tfvars
**/.terraform.lock.hcl
**/crash.log
**/crash.*.log
**/*.tfplan

# Ansible
*.retry

# Generated files
**/output/
**/configs/

# OS files
.DS_Store
*.swp
*.swo
EOF

echo "Created .gitignore:"
cat .gitignore

### 8.2 Initialize Git Repository

In [ ]:
%%bash
cd ~/iac-lab

# Install git if not present
which git || sudo apt-get install -y git

# Initialize repository
git init
git config user.email "lab@devops.local"
git config user.name "DevOps Lab"

# Add files
git add .
git status

In [ ]:
%%bash
cd ~/iac-lab

git commit -m "Initial IaC lab setup with Terraform and Ansible"

echo ""
echo "=== Git Log ==="
git log --oneline

---

## Part 9: Cleanup

### 9.1 Destroy Terraform Resources

In [ ]:
%%bash
cd ~/iac-lab/terraform-ansible-integration/terraform

echo "=== Current Containers ==="
sudo docker ps --filter "label=managed_by=terraform"

echo ""
echo "=== Destroying Terraform Resources ==="
sudo terraform destroy -auto-approve

In [ ]:
%%bash
echo "=== Verify Containers Removed ==="
sudo docker ps --filter "label=managed_by=terraform"
echo "(Should show no containers)"

---

## Summary

In this lab, you learned:

### Terraform
- **HCL Syntax**: Blocks, arguments, and expressions
- **Workflow**: init → plan → apply → destroy
- **Providers**: local, random, docker
- **Variables**: Input variables, tfvars files, variable types
- **Outputs**: Exporting values from configurations
- **State**: How Terraform tracks infrastructure

### Ansible
- **Inventory**: Defining hosts and groups
- **Ad-hoc Commands**: Quick one-off tasks
- **Playbooks**: YAML-based automation
- **Variables**: Defining and using variables
- **Templates**: Jinja2 templating
- **Handlers**: Event-driven tasks

### Integration
- Terraform provisions infrastructure (Docker containers)
- Terraform generates Ansible inventory
- Ansible configures the provisioned resources

### Key Takeaways

1. **Terraform** is for provisioning infrastructure (creating resources)
2. **Ansible** is for configuration management (configuring resources)
3. Together they provide a complete **Infrastructure as Code** solution
4. Version control your infrastructure code just like application code
5. Use `.gitignore` to exclude sensitive files (state, tfvars)

---

## Appendix: Command Reference

### Terraform Commands
```bash
terraform init      # Initialize working directory
terraform plan      # Preview changes
terraform apply     # Apply changes
terraform destroy   # Destroy resources
terraform fmt       # Format code
terraform validate  # Validate syntax
terraform state list        # List resources in state
terraform state show <resource>  # Show resource details
terraform output    # Show outputs
```

### Ansible Commands
```bash
ansible -i inventory all -m ping           # Ping all hosts
ansible -i inventory all -m setup          # Gather facts
ansible -i inventory all -a 'command'      # Run command
ansible-playbook -i inventory playbook.yml # Run playbook
ansible-playbook --check playbook.yml      # Dry run
ansible-playbook -v playbook.yml           # Verbose output
```

---

**Congratulations!** You have completed the Infrastructure as Code lab!